In [1]:
import json
import pandas as pd
import plotly.graph_objects as go

In [8]:
# Load the data from the JSON files
with open('/Users/davidcamperomana/Library/Mobile Documents/com~apple~CloudDocs/TFG/SemEval/data/dev_gold_labels/dev_subtask1_en.json', 'r') as file:
    dev_subtask1_data = json.load(file)

with open('/Users/davidcamperomana/Library/Mobile Documents/com~apple~CloudDocs/TFG/SemEval/fine_tuned_models/ensemble_by_union/original_train/dev_predicted_labels.json', 'r') as file:
    dev_predicted_labels_data = json.load(file)

# Convert JSON data to DataFrame
subtask1_df = pd.DataFrame(dev_subtask1_data)
predicted_labels_df = pd.DataFrame(dev_predicted_labels_data)

subtask1_df

,id,text,labels,link
0,63292,This is why we're free\n\nThis is why we're sa...,[Causal Oversimplification],https://www.facebook.com/SilentmajorityDJT/pho...
1,70419,IF YOU SAY WE'RE IN THE MIDDLE OF A DEADLY PAN...,"[Loaded Language, Name calling/Labeling, Black...",https://www.facebook.com/PatriotFetch/photos/p...
2,63673,AFTER A YEAR OF ZERO EVIDENCE OF RUSSIAN COLLU...,"[Smears, Misrepresentation of Someone's Positi...",https://www.facebook.com/www.russiatruth.co/ph...
3,71297,MY WAY MY WAY,[],https://www.facebook.com/photo/?fbid=850390752...
4,66340,Putin signed a decree to exclude Lyman from Ru...,[Loaded Language],https://www.facebook.com/photo/?fbid=842540057...
...,...,...,...,...
995,70426,CATHING MEXICANS!!!,[Loaded Language],https://www.facebook.com/GovLookout/photos/pb....
996,64176,\I WILL NEVER REGRET MY SUPPORT FOR PRESIDENT ...,"[Smears, Appeal to authority, Glittering gener...",https://www.facebook.com/SilentmajorityDJT/pho...
997,68485,DEMAND KING SALMAN RELEASE CLERIC SHEIKH SALMA...,[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/amnesty/photos/367919...
998,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...","[Smears, Name calling/Labeling, Loaded Language]",https://www.facebook.com/photo/?fbid=234964353...


In [11]:
subtask1_df_copy = subtask1_df.copy()
subtask1_df_copy

In [10]:
subtask1_df = subtask1_df.explode('labels')
subtask1_df

,id,text,labels,link
0,63292,This is why we're free\n\nThis is why we're sa...,Causal Oversimplification,https://www.facebook.com/SilentmajorityDJT/pho...
1,70419,IF YOU SAY WE'RE IN THE MIDDLE OF A DEADLY PAN...,Loaded Language,https://www.facebook.com/PatriotFetch/photos/p...
1,70419,IF YOU SAY WE'RE IN THE MIDDLE OF A DEADLY PAN...,Name calling/Labeling,https://www.facebook.com/PatriotFetch/photos/p...
1,70419,IF YOU SAY WE'RE IN THE MIDDLE OF A DEADLY PAN...,Black-and-white Fallacy/Dictatorship,https://www.facebook.com/PatriotFetch/photos/p...
1,70419,IF YOU SAY WE'RE IN THE MIDDLE OF A DEADLY PAN...,Smears,https://www.facebook.com/PatriotFetch/photos/p...
...,...,...,...,...
997,68485,DEMAND KING SALMAN RELEASE CLERIC SHEIKH SALMA...,Black-and-white Fallacy/Dictatorship,https://www.facebook.com/amnesty/photos/367919...
998,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...",Smears,https://www.facebook.com/photo/?fbid=234964353...
998,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...",Name calling/Labeling,https://www.facebook.com/photo/?fbid=234964353...
998,75006,"\LISTEN TO MY ORDERS, EVIL PUPPET...\n\n... I ...",Loaded Language,https://www.facebook.com/photo/?fbid=234964353...


In [ ]:

# Assuming 'id' is the common column to join both dataframes
merged_df = subtask1_df.merge(predicted_labels_df, on='id')

# Create a list of unique nodes (categories and labels)
labels = [label for sublist in merged_df['labels'] for label in sublist]
unique_labels = list(set(labels))
nodes = unique_labels

# Create a mapping of node names to indices
node_indices = {node: i for i, node in enumerate(nodes)}

# Prepare the source, target, and value lists for the Sankey diagram
source_indices = []
target_indices = []
values = []

for _, row in merged_df.iterrows():
    for label in row['labels']:
        label_index = node_indices[label]
        target_indices.append(label_index)
        values.append(1)  # Assuming each link has a value of 1

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
))

fig.update_layout(title_text="Sankey Diagram", font_size=10)
fig.show()